In [1]:
import os
import random
import skimage.data
from skimage import io
import skimage.transform as skt
import numpy as np
import tensorflow as tf
from tqdm import tqdm
import sys
sys.path.remove('/opt/ros/kinetic/lib/python2.7/dist-packages')
import cv2

In [2]:
# submap_dir = "/home/ysk/catkin_ws/laser_pic_code/pic/regression_benhaun_448_test_with_no_barrier1/submap"
# pic_dir = "/home/ysk/catkin_ws/laser_pic_code/pic/regression_benhaun_448_test_with_no_barrier1/picture"

# # for i in range(4):
# i = 1
# submap1 = cv2.imread(os.path.join(submap_dir,str(1189) + '.pgm'))
# submap1 = cv2.resize(submap1,(448,448))
# submap1 = cv2.transpose(submap1)
# submap1 = cv2.flip(submap1, 1)
# # submap1 = np.rot90(submap1)
# cv2.imshow("submap",submap1)


# pic1 = cv2.imread(os.path.join(pic_dir, '01189.jpg'))
# cv2.imshow("pic",pic1)

# cv2.waitKey(0)
# cv2.destroyAllWindows()
# print(np.shape(submap1))

In [3]:
# train_label_dir = "/home/ysk/database/new_test_dateset/benhuan_test"
# train_pic_dir = "/home/ysk/database/new_test_dateset/benhuan_test/picture"
# train_submap_dir = "/home/ysk/database/new_test_dateset/benhuan_test/submap"

train_label_dir = "/home/ysk/database/new_train_dateset/new_benhuan_train"
train_pic_dir = "/home/ysk/database/new_train_dateset/new_benhuan_train/picture"
train_submap_dir = "/home/ysk/database/new_train_dateset/new_benhuan_train/submap"

train_txt = os.path.join(train_label_dir,"label.txt")
count = len(open(train_txt,'r').readlines()) - 1
file = open(train_txt,'r')
s = file.readlines()

def get_train_data_batch(i, pic_dir=train_pic_dir, label_dir=train_label_dir,submap_dir=train_submap_dir):
    labels = []
    images = []
    
    image1 = skimage.data.imread(os.path.join(pic_dir, s[i].split(";")[0]))
    submap1 = cv2.imread(os.path.join(submap_dir,str(i) + '.pgm'))
    submap1 = cv2.resize(submap1,(448,448))
    submap1 = cv2.transpose(submap1)
    submap1 = cv2.flip(submap1, 1)
    
#     image2 = skimage.data.imread(os.path.join(pic_dir, s[i+1].split(";")[0]))
#     submap2 = cv2.imread(os.path.join(submap_dir,str(i+1) + '.pgm'))
#     submap2 = cv2.resize(submap2,(448,448))
#     submap2 = cv2.transpose(submap2)
#     submap2 = cv2.flip(submap2, 1)
    
    
#     img = cv2.merge([image1,submap1,image2,submap2])
    img = cv2.merge([image1,submap1])
    images.append(img)
    
    label = s[i].split(";")[1:]
    label = list(map(float,label))
    labels.append(label)
    
    return np.asarray(images),np.asarray(labels)
    
num = 0
train_images, train_labels = get_train_data_batch(num)
print(np.shape(train_images))
print(np.shape(train_labels))
print(train_labels)

(1, 448, 448, 6)
(1, 3)
[[ 6.28203890e-02 -2.89475932e-02 -1.73297033e+02]]


In [4]:
with tf.name_scope("input"):
    input_image = tf.placeholder("float", [None, 448,448,6],name='input_image')/255.*2.-1
    y_ = tf.placeholder("float", [None, 3],name='y_') 

def weight_variable(shape):
    initial = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(initial)

def bias_variable(shape):
    initial = tf.constant(0.1, shape = shape)
    return tf.Variable(initial)

def conv2d(x, W):
    return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding="SAME")

def max_pool_2x2(x):
    return tf.nn.max_pool(x, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding="SAME")

#第0层卷积
with tf.name_scope("conv_0"):
    W_conv0 = weight_variable([7, 7, 6, 64])
    b_conv0 = bias_variable([64])
    conv_layer0 = tf.nn.conv2d(input_image, W_conv0, strides=[1,1,1,1], padding="SAME") + b_conv0
#     conv_layer_nor0 = tf.layers.batch_normalization(conv_layer0,training=True)
    
    h_conv0 = tf.nn.leaky_relu(conv_layer0)
with tf.name_scope("max_pool_0"):
    h_pool0 = max_pool_2x2(h_conv0)   #输出变成224*224*64 [None,224,224,64]

# #第一层卷积层
# #参数的前两维为卷积核的维度，第三个参数为当前层的深度，第四个为输出到层的深度
with tf.name_scope("conv_1"):
    W_conv1 = weight_variable([5, 5, 64, 128])
    b_conv1 = bias_variable([128])
    conv_layer1 = conv2d(h_pool0, W_conv1) + b_conv1
#     conv_layer_nor1 = tf.layers.batch_normalization(conv_layer1,training=True)

    h_conv1 = tf.nn.leaky_relu(conv_layer1)
with tf.name_scope("max_pool1"):
    h_pool1 = max_pool_2x2(h_conv1)   #输出变成112*112*64   [None,112,112,64]

#第二层卷积层
with tf.name_scope("conv_2"):
    W_conv2 = weight_variable([5, 5, 128, 192])
    b_conv2 = bias_variable([192])
    conv_layer2 = conv2d(h_pool1, W_conv2) + b_conv2
#     conv_layer_nor2 = tf.layers.batch_normalization(conv_layer2,training=True)

    h_conv2 = tf.nn.leaky_relu(conv_layer2)
with tf.name_scope("max_pool_2"):
    h_pool2 = max_pool_2x2(h_conv2)  #输出变为56*56*192


#第三层卷积层
with tf.name_scope("conv_3"):
    W_conv3 = weight_variable([3, 3, 192, 128])
    b_conv3 = bias_variable([128])
    conv_layer3 = conv2d(h_pool2, W_conv3) + b_conv3
#     conv_layer_nor3 = tf.layers.batch_normalization(conv_layer3,training=True)
    
    h_conv3 = tf.nn.leaky_relu(conv_layer3)
with tf.name_scope("max_pool_3"):
    h_pool3 = max_pool_2x2(h_conv3)   #输出为28*28*128


#第四层卷积层
with tf.name_scope("conv_4"):
    W_conv4 = weight_variable([3, 3, 128, 64])
    b_conv4 = bias_variable([64])
    conv_layer4 = conv2d(h_pool3, W_conv4) + b_conv4
#     conv_layer_nor4 = tf.layers.batch_normalization(conv_layer4,training=True)

    h_conv4 = tf.nn.leaky_relu(conv_layer4)
with tf.name_scope("max_pool_4"):
    h_pool4 = max_pool_2x2(h_conv4)   #输出变为14*14*64


keep_prob = tf.placeholder("float")

#密集连接层0
with tf.name_scope("fc0"):
    W_fc0 = weight_variable([14*14*64, 2048])
    b_fc0 = bias_variable([2048])

    h_pool4_flat = tf.reshape(h_pool4, [-1, 14*14*64])
    
    out_fc0 = tf.matmul(h_pool4_flat, W_fc0) + b_fc0
#     out_nor_fc0 = tf.layers.batch_normalization(out_fc0,training=True)
    
    h_fc0 = tf.nn.leaky_relu(out_fc0)   #输出为96*2048

    #relu函数作用是求出max(h_fc1,0)
    h_fc1_drop0 = tf.nn.dropout(h_fc0, keep_prob)   #输出为96*2048

#密集连接层1
with tf.name_scope("fc1"):
    W_fc1 = weight_variable([2048, 1024])
    b_fc1 = bias_variable([1024])
    
    out_fc1 = tf.matmul(h_fc1_drop0, W_fc1) + b_fc1
#     out_nor_fc1 = tf.layers.batch_normalization(out_fc1,training=True)

    h_fc1 = tf.nn.leaky_relu(out_fc1)   #输出为2048*1024

    #relu函数作用是求出max(h_fc1,0)
    h_fc1_drop1 = tf.nn.dropout(h_fc1, keep_prob)   #输出为2048*1024

#密集连接层2
with tf.name_scope("fc2"):
    W_fc2 = weight_variable([1024, 512])
    b_fc2 = bias_variable([512])
    
    out_fc2 = tf.matmul(h_fc1_drop1, W_fc2) + b_fc2
#     out_nor_fc2 = tf.layers.batch_normalization(out_fc2,training=True)

    h_fc2 = tf.nn.leaky_relu(out_fc2)
    #relu函数作用是求出max(h_fc1,0)
    h_fc1_drop2 = tf.nn.dropout(h_fc2, keep_prob)
    

#输出层
with tf.name_scope("out_layer"):
    W_fc3 = weight_variable([512, 3])
    y_conv = tf.matmul(h_fc1_drop2, W_fc3)

checkpointsPath = "/home/ysk/database/new_train_dateset/new_benhuan_train/l2_nor_4803_6in_3out_checkpoints/"

path = "/home/ysk/database/"
test_result_path = os.path.join(path,'final_test_result','train_l2_nor_4803_6in_3out_test_result.txt')

if os.path.exists(test_result_path):
    os.remove(test_result_path)

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    saver = tf.train.Saver()
    checkPoint = tf.train.get_checkpoint_state(checkpointsPath)
    if checkPoint and checkPoint.model_checkpoint_path:
        saver.restore(sess,checkPoint.model_checkpoint_path)
        print("restored %s" % checkPoint.model_checkpoint_path)
        
    else:
        print("no checkpoint found!")
        
    for i in range(count-2):
        batch_x,batch_y = get_train_data_batch(i)
        batch_y = batch_y*10000
        
        y_pre = sess.run(y_conv,feed_dict={input_image:batch_x, y_:batch_y,keep_prob:1.0})
        
#         print(batch_y[0][2:])
#         print(y_pre[0][2:])
        
        with open(test_result_path,'a') as f:
            for i in range(3):
                f.write(str(batch_y[0][i]) + ',')
            for i in range(2):
                f.write(str(y_pre[0][i]) + ',')
            f.write(str(y_pre[0][2]) + '\n')

        print("real location is:\n",batch_y)
        print("prediction is:\n",y_pre)
        print("\n")
    print("done!")


INFO:tensorflow:Restoring parameters from /home/ysk/database/new_train_dateset/new_benhuan_train/l2_nor_4803_6in_3out_checkpoints/saved_net.ckpt-199000
restored /home/ysk/database/new_train_dateset/new_benhuan_train/l2_nor_4803_6in_3out_checkpoints/saved_net.ckpt-199000
real location is:
 [[ 6.28203890e+02 -2.89475932e+02 -1.73297033e+06]]
prediction is:
 [[ 4.2335693e+02 -7.8721149e+02 -1.6990166e+06]]


real location is:
 [[-4.04042674e+02 -3.38605794e+02 -1.77859856e+06]]
prediction is:
 [[ 2.5238171e+02 -1.5765383e+03 -1.7777989e+06]]


real location is:
 [[-1.44962029e+03 -3.49789385e+02 -1.79931312e+06]]
prediction is:
 [[    2767.5107     2768.5203 -1781067.9   ]]


real location is:
 [[-2.50802167e+03 -3.41654550e+02  1.79723024e+06]]
prediction is:
 [[-5.2464180e+03  1.0406023e+03  1.7378605e+06]]


real location is:
 [[-3.55557483e+03 -3.08319886e+02  1.79731864e+06]]
prediction is:
 [[  -5962.4707   -2726.903  1725156.    ]]


real location is:
 [[-4.66533527e+03 -2.97051979

real location is:
 [[ -66537.1677844     2857.78174205 1785988.93057   ]]
prediction is:
 [[ -69047.75     3239.445 1774076.5  ]]


real location is:
 [[ -67617.5865249     2907.18382378 1786592.8647    ]]
prediction is:
 [[-6.5920719e+04  1.4303345e+03  1.7535476e+06]]


real location is:
 [[ -68702.4334219     2930.25605346 1792072.82979   ]]
prediction is:
 [[ -72437.27      4012.3274 1764673.9   ]]


real location is:
 [[ -69794.0705707     2959.01277348 1790263.63711   ]]
prediction is:
 [[-6.4303523e+04 -1.7250425e+03  1.7651524e+06]]


real location is:
 [[ -70844.3456704     2985.34578523 1788187.06558   ]]
prediction is:
 [[ -70034.86      2135.2173 1751258.1   ]]


real location is:
 [[ -71927.5003387    3029.5903215 1786985.17279  ]]
prediction is:
 [[ -74402.836     3562.7107 1741772.4   ]]


real location is:
 [[ -72995.580072      3062.83413318 1783754.59049   ]]
prediction is:
 [[ -71747.5      -1805.9707 1759490.5   ]]


real location is:
 [[ -74068.9273859     3105.006

real location is:
 [[-130203.774278    31077.7823737 1389590.30236  ]]
prediction is:
 [[-130350.1     32018.256 1368188.5  ]]


real location is:
 [[-131174.393264   31932.159146 1389254.35609 ]]
prediction is:
 [[-127076.      30816.748 1350149.   ]]


real location is:
 [[-132107.784882    32761.5758994 1394588.31277  ]]
prediction is:
 [[-134971.03    28843.773 1381624.8  ]]


real location is:
 [[-133068.512411    33607.1848467 1394547.04406  ]]
prediction is:
 [[-131769.45    29218.842 1384410.4  ]]


real location is:
 [[-134061.651237    34460.8256102 1399773.07676  ]]
prediction is:
 [[-131661.92    30447.566 1371914.9  ]]


real location is:
 [[-135036.938251    35317.4924927 1394520.75024  ]]
prediction is:
 [[-135793.1     34860.598 1369247.4  ]]


real location is:
 [[-136006.511673    36156.3707172 1394087.73297  ]]
prediction is:
 [[-130912.555   31114.09  1379810.   ]]


real location is:
 [[-136993.048553    37008.6345269 1393449.88706  ]]
prediction is:
 [[-130972.15 

real location is:
 [[-195931.262958    93609.7353652 1325004.07599  ]]
prediction is:
 [[-191081.03   82702.26 1301004.6 ]]


real location is:
 [[-196840.541857    94608.9363568 1334010.71682  ]]
prediction is:
 [[-197620.61   81645.45 1309060.5 ]]


real location is:
 [[-197736.782661    95535.5311556 1342463.16113  ]]
prediction is:
 [[-189361.16   78709.43 1307552.6 ]]


real location is:
 [[-198565.243814    96430.2814942 1335448.32521  ]]
prediction is:
 [[-197495.     92274.45 1276372.2 ]]


real location is:
 [[-199420.318403    97397.0396682 1319930.10299  ]]
prediction is:
 [[-198934.69   84331.02 1277568.6 ]]


real location is:
 [[-200240.634257    98392.4923971 1300842.15812  ]]
prediction is:
 [[-197025.75    92382.414 1258159.2  ]]


real location is:
 [[-201075.767591    99529.3185949 1251010.04966  ]]
prediction is:
 [[-199568.14    96979.086 1204772.   ]]


real location is:
 [[-201810.959201  100729.310545 1195976.82988 ]]
prediction is:
 [[-199900.22   94638.95 1164

real location is:
 [[-204695.913255  167561.827479  932792.761822]]
prediction is:
 [[-202617.58  164700.11  908703.5 ]]


real location is:
 [[-204732.702062  168610.702766  931039.574944]]
prediction is:
 [[-206050.6   165267.08  912489.5 ]]


real location is:
 [[-204781.569752  169698.002154  930566.345714]]
prediction is:
 [[-199067.44  163184.56  904225.7 ]]


real location is:
 [[-204808.303574  170795.78216   929934.118652]]
prediction is:
 [[-205291.88  165355.11  913281.25]]


real location is:
 [[-204835.054447  171907.007926  924352.659976]]
prediction is:
 [[-203187.6   166591.66  900261.3 ]]


real location is:
 [[-204860.073608  173025.17716   922991.818248]]
prediction is:
 [[-204192.33  168682.56  886880.8 ]]


real location is:
 [[-204886.212564  174113.552305  922614.107829]]
prediction is:
 [[-205541.67  172424.84  900292.25]]


real location is:
 [[-204914.998146  175196.803693  923069.384573]]
prediction is:
 [[-205990.08  175756.92  891772.  ]]


real location is

...

real location is:
 [[ 205019.285234  225207.088381 -822758.365848]]
prediction is:
 [[ 198606.5   219454.25 -802518.56]]


real location is:
 [[ 205151.376181  224018.776141 -823171.043204]]
prediction is:
 [[ 197543.77  214450.62 -773633.3 ]]


real location is:
 [[ 205281.559231  222815.272652 -831889.463463]]
prediction is:
 [[ 199999.94  211536.78 -803020.8 ]]


real location is:
 [[ 205387.775023  221629.354813 -838375.134598]]
prediction is:
 [[ 198194.02  210655.86 -803336.9 ]]


real location is:
 [[ 205500.714501  220431.49212  -857327.485435]]
prediction is:
 [[ 200191.25  214677.05 -832443.  ]]


real location is:
 [[ 205523.611631  219259.913329 -896891.49248 ]]
prediction is:
 [[ 199046.95  215537.03 -867380.25]]


real location is:
 [[ 205465.607993  217967.057672 -921683.839079]]
prediction is:
 [[ 197719.78  212162.16 -877374.8 ]]


real location is:
 [[ 205414.897075  216659.975645 -916674.284243]]
prediction is:
 [[ 197196.08  209765.84 -868108.3 ]]


real location is

real location is:
 [[ 206569.809724  137977.905614 -882677.930296]]
prediction is:
 [[ 207716.    132805.64 -858889.56]]


real location is:
 [[ 206585.70139   136723.51578  -887276.845143]]
prediction is:
 [[ 202377.12  132722.2  -857234.56]]


real location is:
 [[ 206597.341293  135489.980685 -891008.082099]]
prediction is:
 [[ 205483.62  127843.42 -861013.25]]


real location is:
 [[ 206614.563497  134257.793902 -887685.402752]]
prediction is:
 [[ 202905.67  128319.89 -855889.9 ]]


real location is:
 [[ 206626.557093  133035.936559 -892519.92536 ]]
prediction is:
 [[ 202317.97  129269.2  -863524.44]]


real location is:
 [[ 206616.117724  131878.89494  -902644.566366]]
prediction is:
 [[ 200356.17   128168.266 -869558.1  ]]


real location is:
 [[ 206582.757457  130629.552576 -906336.41084 ]]
prediction is:
 [[ 200747.2    126942.414 -887328.94 ]]


real location is:
 [[ 206544.670925  129536.304132 -901635.778979]]
prediction is:
 [[ 208922.2   125201.47 -876792.1 ]]


real locat

real location is:
 [[  177035.644212     73488.8630697 -1331515.55028  ]]
prediction is:
 [[  173996.5     67411.71 -1299924.6 ]]


real location is:
 [[  176292.596638     72711.2504359 -1328161.91619  ]]
prediction is:
 [[  170388.03     64473.375 -1287751.5  ]]


real location is:
 [[  175513.793145     71925.1756545 -1326387.93735  ]]
prediction is:
 [[  174770.81    67920.3  -1269001.5 ]]


real location is:
 [[  174780.331952     71152.7856984 -1326541.40399  ]]
prediction is:
 [[  167668.2      62606.934 -1287076.2  ]]


real location is:
 [[  173991.137231     70348.9069408 -1322044.29443  ]]
prediction is:
 [[  167006.38    63974.51 -1273174.5 ]]


real location is:
 [[  173205.332523     69498.4382436 -1317898.18357  ]]
prediction is:
 [[  167116.53    58654.5  -1280293.2 ]]


real location is:
 [[  172378.102367    68576.860242 -1311937.92204 ]]
prediction is:
 [[  170374.73    61620.34 -1277645.6 ]]


real location is:
 [[  171611.545486     67742.1650381 -1313331.53211  ]]

real location is:
 [[  125953.50969      21862.1767313 -1359827.97882  ]]
prediction is:
 [[  122325.58     19300.209 -1324239.1  ]]


real location is:
 [[  125080.284861     21060.3387304 -1361123.39662  ]]
prediction is:
 [[  120062.23     23748.625 -1322402.8  ]]


real location is:
 [[  124291.261517     20331.5210734 -1361738.2948   ]]
prediction is:
 [[  122925.6      16896.031 -1331528.6  ]]


real location is:
 [[  123430.217215     19518.0401169 -1361756.92428  ]]
prediction is:
 [[  120469.76     15021.927 -1337207.2  ]]


real location is:
 [[  122603.041919     18729.5151211 -1361298.23096  ]]
prediction is:
 [[  121709.33     19460.129 -1325008.2  ]]


real location is:
 [[  121797.433791     17967.9512934 -1361473.93094  ]]
prediction is:
 [[  118553.62     16644.744 -1316365.   ]]


real location is:
 [[  120912.576241    17126.692143 -1361916.384   ]]
prediction is:
 [[  117898.75     12244.307 -1319775.9  ]]


real location is:
 [[  120043.57728      16309.3448323 -13

real location is:
 [[ 6.60934182e+04  1.54956470e+03 -1.77557710e+06]]
prediction is:
 [[ 6.4516512e+04  1.7088555e+03 -1.7169248e+06]]


real location is:
 [[ 6.50423900e+04  1.51532694e+03 -1.77592629e+06]]
prediction is:
 [[   64907.875      3551.5955 -1726461.1   ]]


real location is:
 [[ 6.39296333e+04  1.47422278e+03 -1.77677746e+06]]
prediction is:
 [[   66064.83       2579.6697 -1724242.2   ]]


real location is:
 [[ 6.28255238e+04  1.44523153e+03 -1.78309454e+06]]
prediction is:
 [[   61170.38       3775.1702 -1740702.4   ]]


real location is:
 [[ 6.16720831e+04  1.42603987e+03 -1.78176408e+06]]
prediction is:
 [[ 6.1378359e+04  6.7476947e+02 -1.7411559e+06]]


real location is:
 [[ 6.05392452e+04  1.39594842e+03 -1.77375515e+06]]
prediction is:
 [[   61073.477     2505.876 -1742452.1  ]]


real location is:
 [[ 5.93807250e+04  1.33258695e+03 -1.76649541e+06]]
prediction is:
 [[   61021.977      3031.4126 -1734459.5   ]]


real location is:
 [[ 5.83324298e+04  1.26236170e+03